In [1]:
import boto3
import pandas as pd
import os
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.parameter import IntegerParameter
from sagemaker.s3 import S3Uploader
from sagemaker.tuner import HyperparameterTuner
from sklearn.model_selection import train_test_split
import requests
import pandas as pd
import json

region = boto3.Session().region_name
sm_client = boto3.client('sagemaker', region_name=region)

In [41]:
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

In [3]:
role = 'AmazonSageMaker-ExecutionRole-20210301T231771'
account_id = 487576569445

In [4]:
role = 'arn:aws:iam::487576569445:role/service-role/AmazonSageMaker-ExecutionRole-20210301T231771'

In [5]:
model_name = 'single-model-fastapi-3'
model_url = 's3://sagemaker-eu-west-1-487576569445/test-sagemaker-train-container-2021-04-01-16-33-24-535/output/model.tar.gz'
container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, 'test-sagemaker-deploy-container')

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'SingleModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container]
)

print("Model Arn: " + create_model_response['ModelArn'])

Model name: single-model-fastapi-3
Model data Url: s3://sagemaker-eu-west-1-487576569445/test-sagemaker-train-container-2021-04-01-16-33-24-535/output/model.tar.gz
Container image: 487576569445.dkr.ecr.eu-west-1.amazonaws.com/test-sagemaker-deploy-container:latest
Model Arn: arn:aws:sagemaker:eu-west-1:487576569445:model/single-model-fastapi-3


In [6]:
endpoint_config_name = 'DEMO-SignleEndpointConfig-4'
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.large',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: DEMO-SignleEndpointConfig-4
Endpoint config Arn: arn:aws:sagemaker:eu-west-1:487576569445:endpoint-config/demo-signleendpointconfig-4


In [7]:
import time

endpoint_name = 'DEMO-SingleModelEndpoint-4'
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

Endpoint name: DEMO-SingleModelEndpoint-4
Endpoint Arn: arn:aws:sagemaker:eu-west-1:487576569445:endpoint/demo-singlemodelendpoint-4


In [37]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

Endpoint Status: InService


In [42]:
import json
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps({
        'Pclass': 3,
        'Sex': 'male',
        'Age': 22.0,
        'Fare': 7.25,
        'Embarked': 'S'
    })
)
json.loads(response['Body'].read())

{'Survived': 0}

In [43]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '7ba1c23e-08cd-4ced-acb5-92a5b09319b0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7ba1c23e-08cd-4ced-acb5-92a5b09319b0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 01 Apr 2021 17:31:18 GMT'},
  'RetryAttempts': 0}}